In [1]:
import numpy as np
from matplotlib import pyplot as plt
plt.style.use(['science', 'grid', 'ieee'])

from endterm import *

In [2]:
dat = np.genfromtxt("esem4fit.txt")
x, y = dat[:, 0], dat[:, 1]

# Sigma values for y - perhaps as least count
yerr = np.tile([0.001], x.shape[0])

In [3]:
# Legendre LSq fit - Order = 4
A, (_, (a0, a1, a2, a3, a4), _) = polyfit(x, y, yerr, n=4, basis="legendre")
_4tuple = (a0, a1, a2, a3, a4)
_4std_tuple = np.sqrt(np.diag(inverse(A)[1]))
# Printing coeffs with their respective standard deviation and relative error
print("Coefficient \t Standard Deviation \t Relative Error (in %)")
for coeff, std in zip(_4tuple, _4std_tuple):
    print(f"{coeff:.5} \t {std:.5} \t\t {np.abs(std / coeff) * 100:.5}")

Coefficient 	 Standard Deviation 	 Relative Error (in %)
0.069658 	 0.00019612 		 0.28154
0.003624 	 0.00032686 		 9.0193
-0.012083 	 0.0004181 		 3.4603
0.011426 	 0.00045977 		 4.0238
0.11049 	 0.00051418 		 0.46535


In [4]:
# Legendre LSq fit - Order = 5
A, (_, (a0, a1, a2, a3, a4, a5), _) = polyfit(x, y, yerr, n=5, basis="legendre")
_5tuple = (a0, a1, a2, a3, a4, a5)
_5std_tuple = np.sqrt(np.diag(inverse(A)[1]))
# Printing coeffs with their respective standard deviation and relative error
print("Coefficient \t Standard Deviation \t Relative Error (in %)")
for coeff, std in zip(_5tuple, _5std_tuple):
    print(f"{coeff:.5} \t {std:.5} \t\t {np.abs(std / coeff) * 100:.5}")

Coefficient 	 Standard Deviation 	 Relative Error (in %)
0.069658 	 0.00019612 		 0.28154
0.0043017 	 0.00032686 		 7.5984
-0.012083 	 0.0004181 		 3.4603
0.013084 	 0.00048225 		 3.6858
0.11049 	 0.00051418 		 0.46535
-0.006727 	 0.00055234 		 8.2109


In [5]:
# Legendre LSq fit - Order = 5
A, (_, (a0, a1, a2, a3, a4, a5, a6), _) = polyfit(x, y, yerr, n=6, basis="legendre")
_6tuple = (a0, a1, a2, a3, a4, a5, a6)
_6std_tuple = np.sqrt(np.diag(inverse(A)[1]))
# Printing coeffs with their respective standard deviation and relative error
print("Coefficient \t Standard Deviation \t Relative Error (in %)")
for coeff, std in zip(_6tuple, _6std_tuple):
    print(f"{coeff:.5} \t {std:.5} \t\t {np.abs(std / coeff) * 100:.5}")

Coefficient 	 Standard Deviation 	 Relative Error (in %)
0.070032 	 0.00019612 		 0.28004
0.0043017 	 0.00032686 		 7.5984
-0.010167 	 0.00043163 		 4.2455
0.013084 	 0.00048225 		 3.6858
0.11412 	 0.00053929 		 0.47257
-0.006727 	 0.00055234 		 8.2109
-0.012385 	 0.00059353 		 4.7925


In [6]:
fig, axs = plt.subplots(1, 3, figsize=(27,8))
fig.suptitle("Plots", fontsize=20)
axs[0].plot(x, func_to_fit(x, _4tuple, basis=legendre), label="Legendre LSQ (Order = 4)")
axs[0].plot(x, y, ".-", label="Original Data")
axs[1].plot(x, func_to_fit(x, _5tuple, basis=legendre), label="Legendre LSQ (Order = 5)")
axs[1].plot(x, y, ".-", label="Original Data")
axs[2].plot(x, func_to_fit(x, _6tuple, basis=legendre), label="Legendre LSQ (Order = 6)")
axs[2].plot(x, y, ".-", label="Original Data")

axs[0].legend()
axs[1].legend()
axs[2].legend()
axs[0].grid()
axs[1].grid()
axs[2].grid()
plt.show()

In [7]:
weights_and_zeros = {
    4: {
        'weights': np.array([0.347854845, 0.652145154, 0.652145154, 0.347854845]),
        'zeros': np.array([0.861136311, 0.339981043, -0.339981043, -0.861136311])
    },
    5: {
        'weights': np.array([0.236926885, 0.478628670, 0.568888889, 0.478628670, 0.236926885]),
        'zeros': np.array([0.906179845, 0.53846931, 0., -0.53846931, -0.906179845])
    },
    6: {
        'weights': np.array([0.171324492, 0.360761573, 0.467913934, 0.467913934, 0.360761573, 0.171324492]),
        'zeros': np.array([0.932469514, 0.661209386, 0.238619186, -0.238619186, -0.661209386, -0.932469514])
    }
}

def gauss_quad(f:callable, lims:tuple, order:int=4):
    a, b = lims
    lim0 = (b - a) / 2
    lim1 = (a + b) / 2

    weights = weights_and_zeros[order]['weights']
    zeros = weights_and_zeros[order]['zeros']

    return lim0 * np.sum(weights * f(lim0 * zeros + lim1))

In [8]:
L = 2 # Length. So, integration limits = (-1, 1)
d = 1 # Distance from the center of the wire
λ = 1 # Uniform charge density
k = 1
lims = (-1, 1)

def φ_int(x):
    """
    Integrand

    """
    return (k * λ) / (np.sqrt(x ** 2 + d ** 2))

def φ_th(d):
    """
    Analytical value of the integral

    """
    return k * λ * np.log((1 + np.sqrt(1 + d ** 2)) / (-1 + np.sqrt(1 + d ** 2)))

In [9]:
# 4-point
φ4 = gauss_quad(φ_int, lims, 4)

# 5-point
φ5 = gauss_quad(φ_int, lims, 5)

# 6-point
φ6 = gauss_quad(φ_int, lims, 6)

# Analytical
φ_ana = φ_th(d=1)

print(f"Results using Gaussian Quadrature at various orders up to 10^(-9) precision:\nOrder = 4: {φ4:.10}\nOrder = 5: {φ5:.10}\nOrder = 6: {φ6:.10}\n")
print(f"Analytical value of the integral for given parameters up to 10^(-9) precision = {φ_ana:.10}.")

Results using Gaussian Quadrature at various orders up to 10^(-9) precision:
Order = 4: 1.762054179
Order = 5: 1.762855295
Order = 6: 1.762730048

Analytical value of the integral for given parameters up to 10^(-9) precision = 1.762747174.


In [11]:
print(f"Relative error at 10^{-9} (shown up to 4 significant digits):\nOrder = 4: {np.abs((φ_ana - φ4) / φ_ana):.4}\nOrder = 5: {np.abs((φ_ana - φ5) / φ_ana):.4}\nOrder = 6: {np.abs((φ_ana - φ6) / φ_ana):.4}\n")

Relative error at 10^-9 (shown up to 4 significant digits):
Order = 4: 0.0003931
Order = 5: 6.134e-05
Order = 6: 9.715e-06

